In [62]:
#Importamos las librerias necesarias
import pandas as pd
import numpy as np
import random as rm
import re

In [63]:
#Empezamos la exploración de los datos cargando los datos en dataframes
actors = pd.read_csv('../ignored/data_raw/actor.csv',encoding='utf-8',encoding_errors= 'ignore')
categories = pd.read_csv('../ignored/data_raw/category.csv',encoding='utf-8',encoding_errors= 'ignore')
films = pd.read_csv('../ignored/data_raw/film.csv',encoding='utf-8',encoding_errors= 'ignore')
inventory = pd.read_csv('../ignored/data_raw/inventory.csv',encoding='utf-8',encoding_errors= 'ignore')
languages = pd.read_csv('../ignored/data_raw/language.csv',encoding='utf-8',encoding_errors= 'ignore')
old = pd.read_csv('../ignored/data_raw/old_HDD.csv',encoding='utf-8',encoding_errors= 'ignore')
rent = pd.read_csv('../ignored/data_raw/rental.csv',encoding='utf-8',encoding_errors= 'ignore')

In [64]:
films.head()

,film_id,title,description,release_year,language_id,original_language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,NaN,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42
1,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,NaN,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42
2,3,ADAPTATION HOLES,A Astounding Reflection of a Lumberjack And a ...,2006,1,NaN,7,2.99,50,18.99,NC-17,"Trailers,Deleted Scenes",2006-02-15 05:03:42
3,4,AFFAIR PREJUDICE,A Fanciful Documentary of a Frisbee And a Lumb...,2006,1,NaN,5,2.99,117,26.99,G,"Commentaries,Behind the Scenes",2006-02-15 05:03:42
4,5,AFRICAN EGG,A Fast-Paced Documentary of a Pastry Chef And ...,2006,1,NaN,6,2.99,130,22.99,G,Deleted Scenes,2006-02-15 05:03:42


In [65]:
#Tras una observación exshaustiva de los datos me doy cuenta de que necesito tener una columna de Category_ID en el df de films


'''

NO LO HAGAS ASI, QUE ES MUCHO MAS INEFICIENTE :)

films['category_id'] = np.NAN

for e in range(len(old['title'])):
    for i in range(len(films['title'])):
        if old['title'][e] == films['title'][i]:
            films['category_id'][i] = old['category_id'][e]
'''
films = pd.merge(films,old, on='title',how = 'left')


In [66]:
#elimino las columnas sobrantes de la suma anterior

films.columns
films.drop(['original_language_id','first_name','last_name','release_year_y'],axis = 1,inplace=True)

In [67]:
films.head()

,film_id,title,description,release_year_x,language_id,rental_duration,rental_rate,length,replacement_cost,rating,special_features,last_update,category_id
0,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,6.0
1,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,6.0
2,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,6.0
3,1,ACADEMY DINOSAUR,A Epic Drama of a Feminist And a Mad Scientist...,2006,1,6,0.99,86,20.99,PG,"Deleted Scenes,Behind the Scenes",2006-02-15 05:03:42,6.0
4,2,ACE GOLDFINGER,A Astounding Epistle of a Database Administrat...,2006,1,3,4.99,48,12.99,G,"Trailers,Deleted Scenes",2006-02-15 05:03:42,11.0
